In [ ]:
% pip install python-dotenv

In [14]:

import os
import requests
from dataclasses import dataclass, asdict
from tqdm import tqdm
import pandas as pd
from bs4 import BeautifulSoup
from collections import Counter
from dotenv import load_dotenv

In [5]:
target_url = "https://github.com/tauri-apps/awesome-tauri"
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")

In [6]:
res = requests.get(target_url)

In [7]:
soup = BeautifulSoup(res.text, 'html.parser')

In [8]:
readme = soup.find('div', id="readme")

In [9]:
a_links = readme.find_all('a')

In [10]:
hrefs = [link.get('href') for link in a_links]

In [11]:
@dataclass
class RepoInfo:
    owner: str
    repo: str
    stars: int
    forks: int
    description: str
    license: str

In [12]:
def https_filter(href: str):
    return 'https' in href

def github_filter(href: str):
    return 'github.com' in href

def is_repo_filter(href: str):
    return len(href.split('/')) == 5

urls = list(filter(is_repo_filter, filter(github_filter, filter(https_filter, hrefs))))

In [13]:
def get_owner_repo(url: str):
    return url.split("/")[-2:]

def get_repo_info(owner: str, repo: str):
    url = f"https://api.github.com/repos/{owner}/{repo}"
    response = requests.get(url, headers={
        "Authorization": f"Bearer {GITHUB_TOKEN}"
    })
    json_data = response.json()
    info = RepoInfo(
        owner=owner,
        repo=repo,
        stars=json_data["stargazers_count"],
        forks=json_data["forks_count"],
        description=json_data["description"],
        license=json_data["license"]["name"] if json_data["license"] else None
    )
    return info

In [15]:
data = []
for url in tqdm(urls):
    owner, repo = get_owner_repo(url)
    try:
        info = get_repo_info(owner, repo)
    except:
        print(f"url failed: {url}")
        info = RepoInfo(
            owner=owner,
            repo=repo,
            stars=0,
            forks=0,
            description="N/A",
            license=""
        )
    data.append(asdict(info))

  0%|          | 0/143 [00:00<?, ?it/s]

 81%|████████  | 116/143 [00:34<00:07,  3.82it/s]

url failed: https://github.com/angeldollface/gluhny


100%|██████████| 143/143 [00:43<00:00,  3.32it/s]


In [16]:
star_df = pd.DataFrame(data=data)
star_df

,owner,repo,stars,forks,description,license
0,tauri-apps,create-tauri-app,770,66,Rapidly scaffold out a new tauri app project.,Apache License 2.0
1,probablykasper,tauri-svelte-template,143,17,Tauri + Svelte template,None
2,oSethoum,tauri-react-template,49,6,"Tauri, React, Vite and Typescript template.",None
3,marc2332,tauri-deno-starter,85,10,"Starter template for Tauri, bundling the front...",None
4,Uninen,tauri-vue-template,121,18,"Tauri + Vue + Vite template w/ Vitest, Tailwin...",None
...,...,...,...,...,...,...
138,cablehead,stacks,8,1,"clip clip, pew pew",MIT License
139,acarl005,toerings,35,1,A clone of Conky Seamod using Tauri,MIT License
140,openscopeproject,TrguiNG,78,3,Remote GUI for Transmission torrent daemon,GNU Affero General Public License v3.0
141,ParthJadhav,verve,510,17,Verve is a lightweight and blazingly fast laun...,GNU Affero General Public License v3.0


In [18]:
sorted_df = star_df.dropna().sort_values(by='stars', ascending=False)

In [28]:
pd.set_option('display.max_rows', 100)
sorted_df.head(100)

,owner,repo,stars,forks,description,license
44,lencx,ChatGPT,43064,5126,"🔮 ChatGPT Desktop Application (Mac, Windows an...",GNU Affero General Public License v3.0
65,spacedriveapp,spacedrive,26798,864,Spacedrive is an open source cross-platform fi...,GNU Affero General Public License v3.0
56,tw93,Pake,19142,2575,🤱🏻 Turn any webpage into a desktop app with Ru...,MIT License
43,Bin-Huang,chatbox,15408,1626,Chatbox is a desktop app for multiple cutting-...,GNU General Public License v3.0
133,pot-app,pot-desktop,6067,269,🌈一个跨平台的划词翻译和OCR软件 | A cross-platform software ...,GNU General Public License v3.0
89,rustdesk,rustdesk-server,4881,987,RustDesk Server Program,GNU Affero General Public License v3.0
69,kimlimjustin,xplorer,3426,270,"Xplorer, a customizable, modern file manager",Apache License 2.0
117,padloc,padloc,2411,222,"A modern, open source password manager for ind...",GNU Affero General Public License v3.0
66,a5huynh,spyglass,2247,48,A personal search engine: Create a searchable...,GNU Affero General Public License v3.0
57,Ironclad,rivet,1924,147,The open-source visual AI programming environm...,MIT License
